
**<h1>프렌즈 영화 스크립트 코퍼스 분석</h1>**




In [ ]:
import json 
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

데이터 로드

In [ ]:
!tar -zxvf /content/drive/MyDrive/고려대빅데이터융합학과/빅데이터처리를위한자연어처리/KU-NLP-2020-1/EmotionLines_friends_annotation.tar.gz

json 데이터 전처리 작업


In [ ]:
a = pd.read_json(r'/content/EmotionLines/Friends/friends_train.json')

In [ ]:
a

In [ ]:
doc_type_list = [k for k in a.keys()]
doc_list =[j for j in a[0].keys()]

In [ ]:
dict_es =pd.DataFrame()
dict_es1=pd.DataFrame(columns=['speaker','utterance','emotion','annotation'])
dict_es2=pd.DataFrame(columns=['speaker','utterance','emotion','annotation'])
for j in doc_list:
    for k in doc_type_list:
           try:
                dict_es[k] =pd.Series(list(a[k][j].values()),index =['speaker','utterance','emotion','annotation'])
                print(dict_es[k])
                dict_es1 = dict_es1.append(dict_es[k],ignore_index=True)
                print(dict_es1)
           except:
                pass
    
    k=0

In [ ]:
dict_es1#한개의 리스트로 정렬

데이터 탐색


In [ ]:
dict_es1['emotion'].value_counts().plot(kind = 'bar')

In [ ]:
print(dict_es1.groupby('emotion').size().reset_index(name = 'count'))

In [ ]:
train_data =dict_es1[['utterance','emotion']]

In [ ]:
print('훈련용 스크립트 개수 :',len(train_data))

데이터 전처리
<p>1.공백 데이터 처리
<p>2.다른 언어 및 특수문자 처리
<p>3.불용어 처리
<p>4.빈도수가 낮은 단어 처리
<p>5.토큰작업 및 벡터화 처리

In [ ]:
train_data.drop_duplicates(subset=['utterance'], inplace=True)

In [ ]:
train_data.loc[train_data.utterance.isnull()]#null check

In [ ]:
train_data = train_data.dropna(how = 'any')

In [ ]:
train_data['utterance'] = train_data['utterance'].str.replace("[^a-zA-Z ]","")

In [ ]:
train_data

In [ ]:
train_data['utterance'].replace('', np.nan, inplace=True)

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords


stopwordss =set(stopwords.words('english'))

In [ ]:
X_train = []
for sentence in train_data['utterance']:
    temp_X = []
    temp_X = nltk.word_tokenize(sentence) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwordss] # 불용어 제거
    X_train.append(temp_X)

In [ ]:
X_train[:3]

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [ ]:
print(tokenizer.word_index)

In [ ]:
threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

In [ ]:
# 전체 단어 개수 중 빈도수 2이하인 단어 개수는 제거.
# 0번 패딩 토큰과 1번 OOV 토큰을 고려하여 +2
vocab_size = total_cnt - rare_cnt + 2
print('단어 집합의 크기 :',vocab_size)

In [ ]:
tokenizer = Tokenizer(vocab_size, oov_token = 'OOV') 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
#X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
X_train

In [ ]:
y_train = np.array(train_data['emotion'])
#y_test = np.array(test_data['label'])

In [ ]:
y_train

In [ ]:
drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]

In [ ]:
# 빈 샘플들을 제거
X_train = np.delete(X_train, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)
print(len(X_train))
print(len(y_train))

In [ ]:
print('스크립트의 최대 길이 :',max(len(l) for l in X_train))
print('스크립트의 평균 길이 :',sum(map(len, X_train))/len(X_train))
plt.hist([len(s) for s in X_train], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

In [ ]:
def below_threshold_len(max_len, nested_list):
  cnt = 0
  for s in nested_list:
    if(len(s) <= max_len):
        cnt = cnt + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (cnt / len(nested_list))*100))

In [ ]:
max_len = 30
below_threshold_len(max_len, X_train)

In [ ]:
X_train = pad_sequences(X_train, maxlen = max_len)

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train1 = le.fit_transform(y_train1)
y_test1 = le.fit_transform(y_test1)
#le.inverse_transform(result)

In [ ]:
from sklearn.model_selection import train_test_split
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_train, y_train, test_size= 0.1, random_state=1234)

모델 학습

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Dense, Input, Flatten, Concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

In [ ]:
embedding_dim = 128
dropout_prob = (0.5, 0.8)
num_filters = 128

In [ ]:
model_input = Input(shape = (max_len,))
z = Embedding(vocab_size, embedding_dim, input_length = max_len, name="embedding")(model_input)
z = Dropout(dropout_prob[0])(z)

In [ ]:
conv_blocks = []

for sz in [3, 4, 5]:
    conv = Conv1D(filters = num_filters,
                         kernel_size = sz,
                         padding = "valid",
                         activation = "relu",
                         strides = 1)(z)
    conv = GlobalMaxPooling1D()(conv)
    conv = Flatten()(conv)
    conv_blocks.append(conv)

In [ ]:
z = Concatenate()(conv_blocks) if len(conv_blocks) > 1 else conv_blocks[0]
z = Dropout(dropout_prob[1])(z)
z = Dense(128, activation="relu")(z)
#z = Dropout(dropout_prob[1])(z)
#z = Dense(128, activation="relu")(z)
model_output = Dense(8, activation="softmax")(z)

model = Model(model_input, model_output)
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["acc"])

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('CNN_model2.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.fit(X_train1, y_train1, batch_size = 64, epochs=10, validation_data = (X_test1, y_test1), verbose=2, callbacks=[es, mc])

In [ ]:
loaded_model = load_model('CNN_model2.h5')#model2 -> 45.51% , model1->45.85%
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test1, y_test1)[1]))

테스트 데이터 예측 작업

In [ ]:
data_final = pd.read_csv('/content/drive/MyDrive/고려대빅데이터융합학과/빅데이터처리를위한자연어처리/en_data.csv')

In [ ]:
data_final['utterance'] = data_final['utterance'].str.replace("[^a-zA-Z ]","")

In [ ]:
def sentiment_predict(new_sentence):
  new_sentence = nltk.word_tokenize(new_sentence) # 토큰화
  new_sentence = [word for word in new_sentence if not word in stopwordss] # 불용어 제거
  encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
  pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
  score = loaded_model.predict(pad_new) # 예측
  y_classes = score.argmax(axis=-1)
  return y_classes

In [ ]:
data_final['Predicted'] = data_final['utterance'].apply(sentiment_predict)

In [ ]:
data_final2['Predicted'] = data_final2['Predicted'].str.replace('[','')

In [ ]:
data_final.to_csv('EN_DATA.csv')

In [ ]:
data_final3=pd.read_csv('/content/drive/MyDrive/고려대빅데이터융합학과/빅데이터처리를위한자연어처리/EN_DATA_TEST1 - EN_DATA_TEST1.csv')

In [ ]:
data_final3['Predicted2'] = le.inverse_transform(data_final3['Predicted'])

In [ ]:
data_final3.to_csv('/content/drive/MyDrive/고려대빅데이터융합학과/빅데이터처리를위한자연어처리/EN_DATA_TEST2.csv',sep=',')